In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [3]:
!unzip '/content/drive/MyDrive/archive.zip' -d '/content'

Streaming output truncated to the last 5000 lines.
  inflating: /content/images_before_edit/x8ks9b.png  
  inflating: /content/images_before_edit/x8lfte.png  
  inflating: /content/images_before_edit/x8lo5v.png  
  inflating: /content/images_before_edit/x8mbla.png  
  inflating: /content/images_before_edit/x8mhgu.png  
  inflating: /content/images_before_edit/x8npp4.png  
  inflating: /content/images_before_edit/x8nr09.png  
  inflating: /content/images_before_edit/x8om0s.png  
  inflating: /content/images_before_edit/x8ozhu.png  
  inflating: /content/images_before_edit/x8rhav.png  
  inflating: /content/images_before_edit/x8tg8v.png  
  inflating: /content/images_before_edit/x8txfm.png  
  inflating: /content/images_before_edit/x8vzvu.png  
  inflating: /content/images_before_edit/x8w2i6.png  
  inflating: /content/images_before_edit/x8yg0l.png  
  inflating: /content/images_before_edit/x8z0gt.png  
  inflating: /content/images_before_edit/x9015n.png  
  inflating: /content/images_be

In [4]:
import os
from pathlib import Path
from google.colab.patches import cv2_imshow

import cv2
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

### Loading the Data

In [ ]:
BATCH_SIZE = 64
IMG_WIDTH = 200
IMG_HEIGHT = 50

In [5]:
data_dir = Path('/content/images_before_edit/')

# downloading all images and converting them into list
images = sorted(list(map(str, list(data_dir.glob("*.png")))))

# getting all labels of images
labels = [img.split(os.path.sep)[-1].split(".png")[0] for img in images]

# getting all characters from labels
characters = set(char for label in labels for char in label)

In [6]:
print(len(images), len(labels))

64961 64961


### All characters for image names

In [7]:
# print all information about characters
digits = sorted(characters)[:10]
letters = sorted(characters)[10:]

print(f"Digits : {digits}")
print(f"Letters : {letters}")

Digits : ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
Letters : ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


### Spliting the Data
- Train Dataset : 80 %
- Validation Dataset : 10 %
- Test Dataset : 10 %

In [8]:
# converting a characters into number 
char_to_num = keras.layers.StringLookup(
    vocabulary=list(characters), mask_token=None
)


# converting vice versa
num_to_char = keras.layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), mask_token=None, invert=True
)


# data distribution
def split_data(images, labels, train_size=0.8, shuffle=True):
    size = len(images)

    # data shuffling
    indices = np.arange(size)
    if shuffle:
        np.random.shuffle(indices)

    samples_80 = int(len(images) * train_size)
    samples_90 = int(len(images) * (train_size + (1 - train_size) / 2))

    x_train, y_train = images[indices[:samples_80]], labels[indices[:samples_80]]
    x_valid, y_valid = images[indices[samples_80:samples_90]], labels[indices[samples_80:samples_90]]
    x_test, y_test = images[indices[samples_90:]], labels[indices[samples_90:]]

    return x_train, x_valid, x_test, y_train, y_valid, y_test


x_train, x_valid, x_test, y_train, y_valid, y_test = split_data(np.array(images), np.array(labels))


# function for image processing and
# for label encoding to 'utf-8'
def encode_single_sample(img_path, label):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_png(img, channels=1)

    img = tf.image.convert_image_dtype(img, tf.float32)

    img = tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])
    img = tf.transpose(img, perm=[1, 0, 2])

    # img /= 255.0 --> normalization

    label = char_to_num(tf.strings.unicode_split(label, input_encoding="UTF-8"))

    return {"image": img, "label": label}

### Data manipulation with _.from_tensor_slices(...)_

In [9]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))

train_dataset = (
    train_dataset.map(
        encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE
    )
    .batch(BATCH_SIZE)
    .prefetch(8) # buffer_size=tf.data.AUTOTUNE
)

validation_dataset = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))

validation_dataset = (
    validation_dataset.map(
        encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE
    )
    .batch(BATCH_SIZE)
    .prefetch(8) # buffer_size=tf.data.AUTOTUNE
)

### Data visualization

In [10]:
# fig, ax = plt.subplots(4, 4, figsize=(20, 10))

# for batch in train_dataset.take(1):
#     images = batch["image"]
#     labels = batch["label"]
#     for i in range(16):
#         img = (images[i] * 255).numpy().astype("uint8")
#         label = tf.strings.reduce_join(num_to_char(labels[i])).numpy().decode("utf-8")
#         ax[i // 4, i % 4].imshow(img[:, :, 0].T, cmap="gray")
#         ax[i // 4, i % 4].set_title(label)
#         ax[i // 4, i % 4].axis("off")

# plt.show()

### OCR Model **CRNN** with **CTC-loss-function**

In [11]:
class CTCLayer(keras.layers.Layer):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.loss_fn = keras.backend.ctc_batch_cost

    def call(self, y_true, y_pred):
        batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
        input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
        label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

        input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
        label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

        loss = self.loss_fn(y_true, y_pred, input_length, label_length)
        self.add_loss(loss)

        return y_pred


def build_model():
    input_img = keras.layers.Input(
        shape=(IMG_WIDTH, IMG_HEIGHT, 1), name="image", dtype="float32"
    )
    labels = keras.layers.Input(name="label", shape=(None,), dtype="float32")

    # 1 Conv2D()
    x = keras.layers.Conv2D(
        32,
        (3, 3),
        activation="relu",
        kernel_initializer="he_normal",
        padding="same",
        name="Conv1",
    )(input_img)
    x = keras.layers.MaxPooling2D((2, 2), name="pool1")(x)

    # 2 Conv2D()
    x = keras.layers.Conv2D(
        64,
        (3, 3),
        activation="relu",
        kernel_initializer="he_normal",
        padding="same",
        name="Conv2",
    )(x)
    x = keras.layers.MaxPooling2D((2, 2), name="pool2")(x)
    
    new_shape = ((IMG_WIDTH // 4), (IMG_HEIGHT // 4) * 64)
    x = keras.layers.Reshape(target_shape=new_shape, name="reshape")(x)
    x = keras.layers.Dense(64, activation="relu", name="dense1")(x)
    x = keras.layers.Dropout(0.2)(x)

    # RNNs
    x = keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True, dropout=0.25))(x)
    x = keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True, dropout=0.25))(x)

    x = keras.layers.Dense(
        len(char_to_num.get_vocabulary()) + 1, activation="softmax", name="dense2"
    )(x)

    output = CTCLayer(name="ctc_loss")(labels, x)

    model = keras.models.Model(
        inputs=[input_img, labels], outputs=output, name="ocr_model_v1"
    )

    opt = keras.optimizers.Adam()
    model.compile(optimizer=opt)

    return model

In [12]:
model = build_model()
model.summary()

Model: "ocr_model_v1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 image (InputLayer)             [(None, 200, 50, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 Conv1 (Conv2D)                 (None, 200, 50, 32)  320         ['image[0][0]']                  
                                                                                                  
 pool1 (MaxPooling2D)           (None, 100, 25, 32)  0           ['Conv1[0][0]']                  
                                                                                                  
 Conv2 (Conv2D)                 (None, 100, 25, 64)  18496       ['pool1[0][0]']       

In [13]:
epochs = 100

In [14]:
history = model.fit(train_dataset, validation_data=validation_dataset, epochs=epochs)

Epoch 1/100
812/812 [==============================] - 107s 108ms/step - loss: 23.4113 - val_loss: 22.5245
Epoch 2/100
812/812 [==============================] - 88s 109ms/step - loss: 22.2756 - val_loss: 21.9290
Epoch 3/100
812/812 [==============================] - 95s 117ms/step - loss: 21.3114 - val_loss: 19.7272
Epoch 4/100
812/812 [==============================] - 87s 107ms/step - loss: 18.2829 - val_loss: 16.2619
Epoch 5/100
812/812 [==============================] - 85s 104ms/step - loss: 14.9497 - val_loss: 12.4283
Epoch 6/100
812/812 [==============================] - 85s 105ms/step - loss: 11.5821 - val_loss: 7.6916
Epoch 7/100
812/812 [==============================] - 86s 106ms/step - loss: 7.0678 - val_loss: 4.7721
Epoch 8/100
812/812 [==============================] - 87s 107ms/step - loss: 5.0268 - val_loss: 3.0728
Epoch 9/100
812/812 [==============================] - 87s 107ms/step - loss: 3.6763 - val_loss: 2.1160
Epoch 10/100
812/812 [==============================

In [ ]:
tf.saved_model.save(model, '/contenct/drive/MyDrive/project-captcha-recognition')